In [ ]:
# default_exp core.rq_database

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *
import json
import responses
from pathlib import Path
from discogspy.core import *

In [ ]:
#hide
with open(Path("../config/example_user_info.json"), "r") as user_config_file:
    config = json.load(user_config_file)
    example_user = UserWithUserTokenBasedAuthentication(user_token=config["user_token"],
                                                        user_agent=config["user_agent"])

# Core Database Requests

> Core.rq_database is a collection of function wrappers around the Discogs api for database interactions.

In [ ]:
#export
import requests
from typing import Union
from discogspy.core import *

## Release

In [ ]:
#export


def get_release(user: Union[UserWithoutAuthentication,
                            UserWithUserTokenBasedAuthentication], 
                release_id: int, 
                curr_abbr: Union[CurrAbbr, None] = None
                ) -> requests.models.Response:
    """
    Get information about a particular release from the Discogs database.
    A release represents a particular physical or digital object released by 
    one or more Artists.
    
    No user Authentication needed.
    
    Parameters
    ----------
    user: user object
        
    release_id : number (required) 
        The Release ID.
        
    curr_abbr: string (optional)
        Currency for marketplace data. Defaults to the authenticated users currency. 
    """
    url = f"{RELEASES_URL}/{release_id}"
    headers = user.headers
    params = user.params
    if curr_abbr: 
        params["curr_abbr"] = curr_abbr.value
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_releases
get_release(user=example_user, release_id=6067651, curr_abbr=CurrAbbr.EUR)

<Response [200]>

In [ ]:
#hide 

@responses.activate
def test_get_release____authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{RELEASES_URL}/123456?curr_abbr=EUR"
    result_content = {"TestRelease": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_release(user=test_user, release_id=123456, curr_abbr=CurrAbbr.EUR)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release____authentication_none()

@responses.activate
def test_get_release___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456?token=test_user_token&curr_abbr=EUR"
    result_content = {"TestRelease": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_release(user=test_user, release_id=123456, curr_abbr=CurrAbbr.EUR)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release___authentication_user_token()

### Release Rating

In [ ]:
#export


def get_user_release_rating(user: Union[UserWithoutAuthentication,
                                        UserWithUserTokenBasedAuthentication], 
                            release_id: int, 
                            username: str
                            ) -> requests.models.Response:
    """
    Get the rating of a release made by the given user.
    
    No user Authentication needed. 
    
    Parameters
    ----------
    user: user object
    
    release_id : number (required) 
        The Release ID.
        
    username: string (required)
        The username of the rating you are trying to request.
    """
    url = f"{RELEASES_URL}/{release_id}/rating/{username}"
    headers = user.headers
    params = user.params
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_user_release_rating
get_user_release_rating(user=example_user, release_id=9453891, username="darkwolf")


<Response [200]>

In [ ]:
#hide 

@responses.activate
def test_get_user_release_rating____authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{RELEASES_URL}/123456/rating/test_user_name"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_user_release_rating(user=test_user, release_id=123456, username="test_user_name")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_user_release_rating____authentication_none()

@responses.activate
def test_get_user_release_rating___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_user_release_rating(user=test_user, release_id=123456, username="test_user_name")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_user_release_rating___authentication_user_token()

In [ ]:
# export


def update_user_release_rating(user: UserWithUserTokenBasedAuthentication, 
                               release_id: int, 
                               username: str,
                               rating: int
                               ) -> requests.models.Response:
    """
    Update the rating of a release made by the given user.
    If there is no rating, it will create one.
    
    User Authentication needed. 
        
    Parameters
    ----------
    user: user object
    
    release_id : number (required) 
        The Release ID.
        
    username: string (required)
        The username of the rating you are trying to request.
        
    rating: int (required) 
        The new rating value. Must be a value between 1 and 5.
        
    """
    url = f"{RELEASES_URL}/{release_id}/rating/{username}"
    headers = user.headers
    params = user.params
    rating = min(max(0, rating), 5)
    data = {"rating": rating}
    return requests.put(url, headers=headers, params=params, json=data)

In [ ]:
# Example for usage of update_user_release_rating

update_user_release_rating(example_user, release_id=11690093, 
                                     username="discogspy", rating=5)

<Response [201]>

In [ ]:
#hide
@responses.activate
def test_update_user_release_rating___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.PUT, target_url,
                  json=result_content, status=201)

    resp = update_user_release_rating(test_user, release_id=123456, 
                                      username="test_user_name", rating=5)
   
    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_update_user_release_rating___authentication_user_token()

@responses.activate
def test_update_user_release_rating_with_rating_higher_5___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.PUT, target_url,
                  json=result_content, status=201)

    resp = update_user_release_rating(test_user, release_id=123456, 
                                      username="test_user_name", rating=6)
   
    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_update_user_release_rating_with_rating_higher_5___authentication_user_token()

@responses.activate
def test_update_user_release_rating_with_rating_lower_0___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.PUT, target_url,
                  json=result_content, status=201)

    resp = update_user_release_rating(test_user, release_id=123456, 
                                      username="test_user_name", rating=-5)
   
    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_update_user_release_rating_with_rating_lower_0___authentication_user_token()

In [ ]:
# export


def delete_user_release_rating(user: UserWithUserTokenBasedAuthentication, 
                               release_id: int, 
                               username: str
                               ) -> requests.models.Response:
    """
    Delete the rating of a release made by the given user.
    
    User Authentication needed. 
    
    Parameters
    ----------
    user: user object
    
    release_id : number (required) 
        The Release ID.
        
    username: string (required)
        The username of the rating you are trying to delete.
        
    """
    url = f"{RELEASES_URL}/{release_id}/rating/{username}"
    headers = user.headers
    params = user.params
    return requests.delete(url, headers=headers, params=params)

In [ ]:
# Example for usage of delete_user_release_rating

delete_user_release_rating(example_user, release_id=11690093, username="discogspy")

<Response [204]>

In [ ]:
#hide

@responses.activate
def test_delete_user_release_rating___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {}
    
    responses.add(responses.DELETE, target_url,
                  json=result_content, status=204)

    resp = delete_user_release_rating(user=test_user, release_id=123456, 
                                      username="test_user_name")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_delete_user_release_rating___authentication_user_token()

In [ ]:
# export


def get_community_release_rating(user: Union[UserWithoutAuthentication,
                                             UserWithUserTokenBasedAuthentication], 
                                 release_id: int
                                 ) -> requests.models.Response:
    """
    Get the rating of a release made by the community.
    A community release rating includes the average rating and 
    the total number of user ratings for a given release.
    
    This function doesn't work for master releases!
    
    No user Authentication needed. 
    
    Parameters
    ----------
    user: user object
    
    release_id : number (required) 
        The Release ID.
       
    """
    url = f"{RELEASES_URL}/{release_id}/rating"
    headers = user.headers
    params = user.params
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_community_release_rating

get_community_release_rating(example_user, release_id=1995816)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_community_release_rating___authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{RELEASES_URL}/123456/rating"
    result_content = {"rating": {"count": 19, "average": 4.89}, "release_id": 1995816}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_community_release_rating(user=test_user, release_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_community_release_rating___authentication_none()

@responses.activate
def test_get_community_release_rating___authentication_user_token():

    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{RELEASES_URL}/123456/rating?token=test_user_token"
    result_content = {"rating": {"count": 19, "average": 4.89}, "release_id": 1995816}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_community_release_rating(user=test_user, release_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_community_release_rating___authentication_user_token()

### Master Release

In [ ]:
#export


def get_master_release(user: Union[UserWithoutAuthentication,
                                   UserWithUserTokenBasedAuthentication], 
                       master_id: int
                       ) -> requests.models.Response:
    """
    Get information to a particular master release from Discogs database.
    A Master release represents a set of similar Releases. 
    Masters releases have a “main release” which is often the chronologically earliest.
    
    No user Authentication needed.
    
    Parameters
    ----------
    user: user object
    
    master_id : number (required) 
        The Master ID.

    """
    url = f"{MASTERS_URL}/{master_id}"
    headers = user.headers
    params = user.params
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_master_release

get_master_release(example_user, master_id=195663)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_master_release___authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{MASTERS_URL}/123456"
    result_content = {"TestMaster": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_master_release(user=test_user, master_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_master_release___authentication_none()

@responses.activate
def test_get_master_release___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{MASTERS_URL}/123456?token=test_user_token"
    result_content = {"TestMaster": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_master_release(user=test_user, master_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_master_release___authentication_user_token()

In [ ]:
#export


def get_releases_related_to_master_release(user: Union[UserWithoutAuthentication,
                                                       UserWithUserTokenBasedAuthentication], 
                                           master_id: int,
                                           page: Union[int, None] = None,
                                           per_page: Union[int, None] = None,
                                           release_format: Union[str, None] = None,
                                           label: Union[str, None] = None,
                                           released: Union[str, None] = None, 
                                           country: Union[str, None] = None,
                                           sort: Union[SortOptionsMaster, None] = None, 
                                           sort_order: Union[SortOrder, None] = None
                                           ) -> requests.models.Response:
    """
    Get a list of all Releases that are versions of the given master release.
    
    No user Authentication needed.
    
    Parameters
    ----------
    user: user object.
    
    master_id : number (required) 
        The Master ID.
    
    page: number (optional) 
        The page you want to request.
        
    per_page: number (optional)
        The number of items per page.
        
    release_format: string (optional)
        The format to filter.
        
    label: string (optional)
        The label to filter.
        
    released: string (optional)
        The release year to filter.
        
    country: string (optional)
        The country to filter.
        
    sort: string (optional)
        Sort items by this field.
        
    sort_order: string (optional)
        Sort items in a particular order (one of asc, desc)
    """

    url = f"{MASTERS_URL}/{master_id}/versions"
    headers = user.headers
    params = user.params
    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)
    if release_format:
        params["format"] = release_format
    if label:
        params["label"] = label
    if released:
        params["released"] = released
    if country:
        params["country"] = country
    if sort:
        params["sort"] = sort.value
    if sort_order:
        params["sort_order"] = sort_order.value
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_releases_related_to_master_release

get_releases_related_to_master_release(example_user, master_id=57960, 
                                       release_format="Cassette", label="Roadrunner Records")

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_releases_related_to_master_release___authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{MASTERS_URL}/123456/versions?page=1&per_page=50&format=CD&label=label&" \
                 f"released=1989&country=country&sort=released&sort_order=asc"
    
    result_content = {"TestMaster": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_releases_related_to_master_release(user=test_user, master_id=123456,
                                                  page=1, per_page=50, release_format="CD",
                                                  label="label", released="1989",
                                                  country="country", sort=SortOptionsMaster.released,
                                                  sort_order=SortOrder.asc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_releases_related_to_master_release___authentication_none()

@responses.activate
def test_get_releases_related_to_master_release___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{MASTERS_URL}/123456/versions?token=test_user_token&page=1&per_page=50&" \
                 f"format=CD&label=label&released=1989&country=country&sort=released&sort_order=asc"
    result_content = {"TestMaster": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_releases_related_to_master_release(user=test_user, master_id=123456,
                                                  page=1, per_page=50, release_format="CD",
                                                  label="label", released="1989",
                                                  country="country", sort=SortOptionsMaster.released,
                                                  sort_order=SortOrder.asc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_releases_related_to_master_release___authentication_user_token()

## Artist

An Artist represents a person in the Discogs database who contributed to a Release in some capacity.

In [ ]:
#export


def get_artist(user: Union[UserWithoutAuthentication,
                           UserWithUserTokenBasedAuthentication], 
               artist_id: int
               ) -> requests.models.Response:
    """
    Get information about an artist.
     
    No user Authentication needed. 
    
    Parameters
    ----------
    user: user object.
    
    artist_id : number (required) 
        The Artist ID.
        
    """
    url = f"{ARTIST_URL}/{artist_id}"
    headers = user.headers
    params = user.params
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_artist
get_artist(user=example_user, artist_id=1516559)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_artist___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{ARTIST_URL}/123456"
    
    result_content = {"TestArtist": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_artist(user=test_user, artist_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_artist___authentication_none()

@responses.activate
def test_get_artist___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{ARTIST_URL}/123456?token=test_user_token"
    
    result_content = {"TestArtist": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_artist(user=test_user, artist_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_artist___authentication_user_token()

In [ ]:
#export


def get_artist_releases(user: Union[UserWithoutAuthentication,
                                    UserWithUserTokenBasedAuthentication], 
                        artist_id: int,
                        page: Union[int, None] = None,
                        per_page: Union[int, None] = None,
                        sort: Union[SortOptionsArtist, None] = None,
                        sort_order: Union[SortOrder, None] = None
                        ) -> requests.models.Response:
    """
    Get a list of releases and masters associated with the given artist.
    
    No user Authentication needed. 
    
    Parameters
    ----------
    user: user object.
    
    artist_id : number (required) 
        The Artist ID.
    
    page: number (optional) 
        The page you want to request.
        
    per_page: number (optional)
        The number of items per page.
        
    sort: string (optional)
        Sort items by this field.
        
    sort_order: string (optional)
        Sort items in a particular order (one of asc, desc)
    """
    url = f"{ARTIST_URL}/{artist_id}/releases"
    headers = user.headers
    params = user.params
    
    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)
    if sort:
        params["sort"] = sort.value
    if sort_order:
        params["sort_order"] = sort_order.value
        
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_artist_releases
get_artist_releases(user=example_user, artist_id=1516559, page=1, per_page=3, 
                    sort=SortOptionsArtist.year, sort_order=SortOrder.desc)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_artist_releases___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{ARTIST_URL}/123456/releases?page=1&per_page=3&sort=year&sort_order=desc"
    
    result_content = {"TestArtist": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_artist_releases(user=test_user, artist_id=123456, page=1, 
                               per_page=3, sort=SortOptionsArtist.year, sort_order=SortOrder.desc)


    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_artist_releases___authentication_none()

@responses.activate
def test_get_artist_releases___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{ARTIST_URL}/123456/releases?token=test_user_token&page=1&per_page=3&sort=year&sort_order=desc"
    
    result_content = {"TestArtist": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_artist_releases(user=test_user, artist_id=123456, page=1, 
                               per_page=3, sort=SortOptionsArtist.year, sort_order=SortOrder.desc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_artist_releases___authentication_user_token()

## Label

A label resource represents a label, company, recording studio, location, or other entity involved with artists and releases. 

In [ ]:
#export


def get_label(user: Union[UserWithoutAuthentication,
                          UserWithUserTokenBasedAuthentication], 
              label_id: int
              ) -> requests.models.Response:
    """
    Get information about a label.
    
    No user Authentication needed. 
    
    Parameters
    ----------
    user: user object.
    
    label_id : number (required) 
        The Label ID.

    """
    url = f"{LABEL_URL}/{label_id}"
    headers = user.headers
    params = user.params
        
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_label
get_label(user=example_user, label_id=18263)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_label___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{LABEL_URL}/123456"
    
    result_content = {"TestLabel": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_label(user=test_user, label_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_label___authentication_none()

@responses.activate
def test_get_label___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{LABEL_URL}/123456?token=test_user_token"
    
    result_content = {"TestLabel": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_label(user=test_user, label_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_label___authentication_user_token()

In [ ]:
#export


def get_label_releases(user: Union[UserWithoutAuthentication,
                                   UserWithUserTokenBasedAuthentication], 
                       label_id: int,
                       page: Union[int, None] = None,
                       per_page: Union[int, None] = None,
                       sort: Union[SortOptionsLabel, None] = None,
                       sort_order: Union[SortOrder, None] = None
                       ) -> requests.models.Response:
    """
    Get a list of releases and masters associated with the given label.
    
    No user Authentication needed. 
    
    Parameters
    ----------
    user: user object.
    
    label_id : number (required) 
        The Label ID.
    
    page: number (optional) 
        The page you want to request.
        
    per_page: number (optional)
        The number of items per page.
        
    sort: string (optional)
        Sort items by this field.
        
    sort_order: string (optional)
        Sort items in a particular order (one of asc, desc)
    """
    url = f"{LABEL_URL}/{label_id}/releases"
    headers = user.headers
    params = user.params
    
    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)
    if sort:
        params["sort"] = sort.value
    if sort_order:
        params["sort_order"] = sort_order.value
        
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_label_releases
get_label_releases(user=example_user, label_id=18263, page=1, per_page=3, 
                   sort=SortOptionsLabel.year, sort_order=SortOrder.desc)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_label_releases___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{LABEL_URL}/123456/releases?page=1&per_page=3&sort=year&sort_order=desc"
    
    result_content = {"TestLabel": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_label_releases(user=test_user, label_id=123456, page=1, 
                              per_page=3, sort=SortOptionsLabel.year, sort_order=SortOrder.desc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_label_releases___authentication_none()

@responses.activate
def test_get_label_releases___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{LABEL_URL}/123456/releases?token=test_user_token&page=1&per_page=3&sort=year&sort_order=desc"
    
    result_content = {"TestLabel": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_label_releases(user=test_user, label_id=123456, page=1, 
                              per_page=3, sort=SortOptionsLabel.year, sort_order=SortOrder.desc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_label_releases___authentication_user_token()

In [ ]:
#hide

from nbdev.export import *
notebook2script()

Converted 00_core.constants.ipynb.
Converted 01_core.discogs_user.ipynb.
Converted 02_core.rq_database.ipynb.
Converted 03_core.rq_marketplace.ipynb.
Converted index.ipynb.
